In [ ]:
import xarray as xr
import xnemogcm as xn
from matplotlib import pyplot as plt
import cftime._cftime as cft
import dask.config

from pathlib import Path

#plt.rcParams['axes.facecolor']='black'

#Setting to true avoids too large chuncks
dask.config.set({'array.slicing.split_large_chunks': True})

#plt.style.use('dark_background')
#plt.rcParams.update({'font.size': 32})

In [ ]:
datadir   = Path("/data/dkamm/nemo_output/NEVERWORLD/1_deg_GM/restart5")

In [ ]:
xr.open_dataset(datadir + 'NEVERWORLD_02880000_restart_0000.nc')

In [ ]:


domcfg = xn.open_domain_cfg(datadir=datadir)

In [ ]:
domcfg

In [ ]:
ds = xn.open_nemo(domcfg=domcfg, datadir=datadir)

In [ ]:
ds.uoce.isel(z_c=0,t=0).plot()

In [ ]:
ds = xn.open_nemo_and_domain_cfg(nemo_files=datadir, domcfg_files=datadir)

In [ ]:
t_star = ds.qns + ds.qsr #+ ds.toce.isel(z_c=0)

In [ ]:
ds.toce.isel(z_c=1, t=-1).isnull().compute().sum()

In [ ]:
ds.toce.isel(z_c=0, t=-1)

In [ ]:
ds.toce.isel(x_c=20, y_c=40).plot()

In [ ]:
ds.toce